In [11]:
import random
import networkx as nx
import matplotlib.pyplot as plt
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import NetworkGrid

IN-degree weighted by engagement.
OUT-degree normalized each step.

Unfollow someone if IN-degree to low compared to out-degree, i.e. not enough engagement received to participate in network.

Follow candidate if they have sufficiently high IN-degree, because chance of backfollow?

Not follow candidate if chance of backfollow is not lucrative enough.

In [12]:
class SocialNetworkAgent(Agent):
    """Agent class models agent behavior individually"""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.engagement_threshold = random.uniform(0, 1)
        self.strategies = [self.make_connection, self.get_recommendation, self.get_recommended]
        self.strategy_weights = [1/3, 1/3, 1/3] # Maybe randomize this according to some distribution
        self.strategy_choice = self.choose_strategy()
        self.edges = list(model.G.edges(unique_id, data=True))

    def check_connected(self) -> bool:
        """ function that checks if a node still has at least one follower
        or at least follows someone (so basically if it is connected) 
        if not, it does not do any of the actions."""
        return len(self.edges) > 1

    def targeted_recommendation(self):
        """Calculate targeted recommendation based on engagement similarity and shortest path length"""
        G = self.model.G
        follower_node = self.unique_id

        # Calculate recommendation scores
        recommendation_scores = {}
        for influencer_node in G.nodes():
            if influencer_node != follower_node:
                try:
                    shortest_path_length = nx.shortest_path_length(G, source=follower_node, target=influencer_node)
                except nx.NetworkXNoPath:
                    continue  # Skip this influencer if no path exists

                follower_engagements = [G[neighbor][follower_node]['engagement'] for neighbor in G.predecessors(follower_node)]
                influencer_engagements = [G[neighbor][influencer_node]['engagement'] for neighbor in G.predecessors(influencer_node)]
                
                # Calculate homogeneity over follower vs influencer
                if follower_engagements and influencer_engagements:
                    mean_follower_engagement = sum(follower_engagements) / len(follower_engagements)
                    mean_influencer_engagement = sum(influencer_engagements) / len(influencer_engagements)
                    homogeneity = 1 - abs(mean_follower_engagement - mean_influencer_engagement)

                    # Use a combination of shortest_path_length and homogeneity for recommendation score
                    recommendation_score = shortest_path_length * homogeneity
                    recommendation_scores[influencer_node] = recommendation_score

        # Normalize recommendation scores
        total_score = sum(recommendation_scores.values())
        normalized_scores = {k: v / total_score for k, v in recommendation_scores.items()} if total_score > 0 else {}

        if normalized_scores:
            chosen_influencer = random.choices(list(normalized_scores.keys()), weights=list(normalized_scores.values()), k=1)
        else:
            chosen_influencer = None

        return chosen_influencer

    # strategies
    ##############################################################################################################
    def make_random_connection(self):
        """Connects node to random node"""
        possible_targets = [node for node in self.model.G.nodes if node != self.unique_id]
        target = random.choice(possible_targets)
        if not self.model.G.has_edge(self.unique_id, target):
            self.model.add_directed_connection(self.unique_id, target)

    def make_connection(self):
        """Makes connection with neighbor of neighbor node if not yet present"""
        neighbors = list(self.model.G.neighbors(self.unique_id))
        potential_targets = set()
        for neighbor in neighbors:
            potential_targets.update(self.model.G.neighbors(neighbor))

        potential_targets = list(potential_targets)
        if potential_targets:
            target = random.choice(potential_targets)
            if not self.model.G.has_edge(self.unique_id, target):
                self.model.add_directed_connection(self.unique_id, target)
            else:
                self.make_random_connection()
        else:
            self.make_random_connection()

    def get_recommendation(self):
        """Requests a recommendation and updates connections with certain loss.

        Chooses an influencer based on a recommendation score. If an influencer is chosen
        and the agent has incoming edges, replaces existing connections with the influencer
        if the new recommendation offers higher engagement. """
        
        chosen_influencer = self.targeted_recommendation()
        if chosen_influencer:
            chosen_influencer = chosen_influencer[0]
            in_edges = list(self.model.G.in_edges(self.unique_id, data=True))
            if in_edges:
                for edge in in_edges:
                    if self.model.G.has_edge(chosen_influencer, self.unique_id):
                        if edge[2]['engagement'] < self.model.G[chosen_influencer][self.unique_id]['engagement']:
                            self.model.G.remove_edge(edge[0], self.unique_id)
                            self.model.G.add_edge(chosen_influencer, self.unique_id, engagement=random.random())
                            break

    def get_recommended(self):
        """Requests a recommendation and updates connections with random loss.
        
        Chooses an influencer based on a recommendation score. If an influencer is chosen
        and the agent has incoming edges, replaces the edge with the lowest engagement
        with a new connection to the influencer with a random engagement value."""
        
        chosen_influencer = self.targeted_recommendation()
        if chosen_influencer:
            chosen_influencer = chosen_influencer[0]
            in_edges = list(self.model.G.in_edges(self.unique_id, data=True))
            if in_edges:
                lowest_engagement_edge = min(in_edges, key=lambda x: x[2]['engagement'])
                if random.random() < self.engagement_threshold:
                    self.model.G.remove_edge(lowest_engagement_edge[0], self.unique_id)
                    self.model.G.add_edge(chosen_influencer, self.unique_id, engagement=random.random())
    ##############################################################################################################
    
    # choose a strategy
    def choose_strategy(self):
        return random.choices(self.strategies, self.strategy_weights)[0]

    # execute chosen strategy
    def step(self):
        if self.check_connected():
            self.strategy_choice()

    # IDEA TO UPDATE WEIGHTS THOUGH PROBABLY TOO INTENSIVE
    def update_weights(self):
        return

In [18]:
class SocialNetworkModel(Model):
    """Social network model class models the interaction between agents"""
    def __init__(self, num_agents, initial_p, interaction="random", option=1):
        super().__init__()
        self.num_agents = num_agents
        self.schedule = RandomActivation(self)
        self.initial_p = initial_p

        self.create_random_network()
                
        # Check if graph is strongy connected, if not take the strongly connected part and use as total
        if not nx.is_strongly_connected(self.G):
            components = list(nx.strongly_connected_components(self.G))
            largest_component = max(components, key=len)
            self.G = self.G.subgraph(largest_component).copy()
        
        # Initialize grid using mesa.space.NetworkGrid
        self.grid = NetworkGrid(self.G)
        
        # Add agents in each node, note each agent has node=unique_id
        for i, node in enumerate(self.G.nodes()):
            agent = SocialNetworkAgent(node, self)
            self.schedule.add(agent)
            self.grid.place_agent(agent, node)
            self.G.nodes[node]['agent'] = agent  # Store the agent in the node attributes
            

        self.data_collector = {"avg_out_degree": [], "avg_in_degree": [],
                               "avg_clustering_coeff": [], "avg_path_length": [],
                               "homogeneity": []}

    def create_random_network(self):
        """Initiate random network with bi-directed edges including 
        random engagement value U(0,1)"""
        self.G = nx.DiGraph()
        undirected_G = nx.erdos_renyi_graph(n=self.num_agents, p=self.initial_p)
        for u, v in undirected_G.edges():
            # Add directed edges when initializing graph, engagement value in each direction
            self.G.add_edge(u, v, engagement=random.random())
            self.G.add_edge(v, u, engagement=random.random())
        # nx.draw(self.G)
        # plt.show()

    def add_directed_connection(self, from_node, to_node):
        """Add directed connections between nodes"""
        if not self.G.has_edge(from_node, to_node):
            self.G.add_edge(from_node, to_node, engagement=random.random())

    def get_mean_in_degree(self):
        """Calculate the mean in-degree of network"""
        degrees = [val for (node, val) in self.G.in_degree()]
        mean_degree = sum(degrees) / len(degrees)
        return mean_degree

    def track_metrics(self):
        """Tracking metrics over network"""
        avg_out_degree = sum(dict(self.G.out_degree()).values()) / self.num_agents
        avg_in_degree = sum(dict(self.G.in_degree()).values()) / self.num_agents
        avg_clustering_coeff = nx.average_clustering(self.G)
        avg_path_length = nx.average_shortest_path_length(self.G)
        homogeneity = self.calculate_homogeneity()

        self.data_collector["avg_out_degree"].append(avg_out_degree)
        self.data_collector["avg_in_degree"].append(avg_in_degree)
        self.data_collector["avg_clustering_coeff"].append(avg_clustering_coeff)
        self.data_collector["avg_path_length"].append(avg_path_length)
        self.data_collector["homogeneity"].append(homogeneity)

    def calculate_homogeneity(self):
        """Calculates homogeneity based on engagement similarity over network"""
        total_similarity = 0
        count = 0
        for node in self.G.nodes():
            engagement_values = [self.G[neighbor][node]['engagement'] for neighbor in self.G.predecessors(node)]
            if len(engagement_values) > 1:
                mean_engagement = sum(engagement_values) / len(engagement_values)
                similarity = sum([1 - abs(engagement - mean_engagement) for engagement in engagement_values]) / len(engagement_values)
                total_similarity += similarity
                count += 1
        return total_similarity / count if count > 0 else 0

    def prune_network(self):
        """Prunes network by modulo of number of agents 
        (if pruned by a percentage, network dies out very soon)"""
        edges = list(self.G.edges())
        num_edges_to_remove = len(edges) // self.num_agents
        edges_to_remove = random.sample(edges, num_edges_to_remove)
        self.G.remove_edges_from(edges_to_remove)
        

    def step(self):
        """Steps in the simulation"""
        if not nx.is_strongly_connected(self.G):
            components = list(nx.strongly_connected_components(self.G))
            largest_component = max(components, key=len)
            removed_nodes = set(self.G.nodes()) - set(largest_component)
            for node in removed_nodes:
                agent = self.G.nodes[node]['agent']
                self.schedule.remove(agent)
            self.G = self.G.subgraph(largest_component).copy()
        self.schedule.step()
        self.track_metrics()
        self.prune_network()

In [19]:
model = SocialNetworkModel(num_agents=100, initial_p=0.5)
num_steps = 500
for i in range(num_steps):
    model.step()
    pos = nx.spring_layout(model.G)
    print(f"\r{(i/num_steps)*100:.2f}%", end='', flush=True)


52.00%

NetworkXError: Graph is not strongly connected.

In [ ]:
import matplotlib.pyplot as plt

# Create a figure with 4 subplots
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

# Plot Avg Out Degree and Avg In Degree together in the first subplot
axs[0, 0].plot(model.data_collector["avg_out_degree"], label="Avg Out Degree")
axs[0, 0].plot(model.data_collector["avg_in_degree"], label="Avg In Degree")
axs[0, 0].set_title("Average Degrees")
axs[0, 0].set_xlabel("Time Steps")
axs[0, 0].set_ylabel("Degree")
axs[0, 0].legend()

# Plot Avg Clustering Coefficient in the second subplot
axs[0, 1].plot(model.data_collector["avg_clustering_coeff"], label="Avg Clustering Coeff", color='green')
axs[0, 1].set_title("Average Clustering Coefficient")
axs[0, 1].set_xlabel("Time Steps")
axs[0, 1].set_ylabel("Clustering Coefficient")
axs[0, 1].legend()

# Plot Avg Path Length in the third subplot
axs[1, 0].plot(model.data_collector["avg_path_length"], label="Avg Path Length", color='red')
axs[1, 0].set_title("Average Path Length")
axs[1, 0].set_xlabel("Time Steps")
axs[1, 0].set_ylabel("Path Length")
axs[1, 0].legend()

# Plot Homogeneity in the fourth subplot
axs[1, 1].plot(model.data_collector["homogeneity"], label="Homogeneity", color='purple')
axs[1, 1].set_title("Homogeneity")
axs[1, 1].set_xlabel("Time Steps")
axs[1, 1].set_ylabel("Homogeneity")
axs[1, 1].legend()

# Adjust the layout to prevent overlapping
plt.tight_layout()

# Show the plot
plt.show()